## Import library

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import data

In [10]:
train=pd.read_csv('titanic_traning.csv')
train.drop(columns='ID',inplace=True)

In [5]:
train.columns

Index(['ID', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked',
       'survived'],
      dtype='object')

In [ ]:
feat_list=train.columns[[]]